In [1]:
import pandas as pd


from constants import okr_excel_path, researcher_info_excel_path, salesperson_info_excel_path, output_excel_path
from utils import get_df_roadshow, get_sales_name_list, get_sales_name_list, write_dfs_to_excel
from utils import print_na_rate, get_df_researcher

In [2]:
df_okr = pd.read_excel(okr_excel_path, sheet_name='路演')
df_researcher_info = pd.read_excel(researcher_info_excel_path, usecols='A:C')
df_salespeople_info =  pd.read_excel(salesperson_info_excel_path)

In [3]:
sales_name_list = get_sales_name_list(df_salespeople_info)

In [4]:
df_roadshow, df_special = get_df_roadshow(df_okr, sales_name_list)
df_researcher = get_df_researcher(df_roadshow)

In [6]:
# # Drop the region column
# df_without_region = df_researcher.drop(columns=['参与地区', '研究员'])

# # Group by and calculate sum, median, and mode
# sum_df = df_without_region.groupby('所属团队').sum().reset_index()
# median_df = df_without_region.groupby('所属团队').median().reset_index()
# mode_df = df_without_region.groupby('所属团队').agg(lambda x: x.mode().iloc[0]).reset_index()

# # Merging dataframes
# df_team = sum_df.merge(median_df, on='所属团队', suffixes=('_sum', '_median'))
# for col in df_team.columns:
#     if not col.endswith(('sum', 'median', '所属团队')):
#         df_team[col + '_mode'] = mode_df[col]

# # Rename the '所属团队' column
# df_team.rename(columns={'所属团队': '行业'}, inplace=True)




In [7]:
dfs_dict = {
            'roadshow': df_roadshow,
            'special': df_special,
            'researcher': df_researcher
            }


write_dfs_to_excel(dfs_dict, okr_excel_path)